In [2]:
import numpy as np

# 色定義（赤=0, 青=1, 黄=2）
color_map = {'赤': 0, '青': 1, '黄': 2}
panel_names = ['A', 'B', 'C', 'D', 'E', 'F']

# 変化するのは両隣のみ（踏んだパネル自身は変化しない）
neighbors = [
    [1, 5],  # A
    [2, 0],  # B
    [3, 1],  # C
    [4, 2],  # D
    [5, 3],  # E
    [0, 4],  # F
]

# 影響行列Aを構築
A = np.zeros((6, 6), dtype=int)
for i in range(6):
    for j in neighbors[i]:
        A[j][i] += 1
A %= 3  # mod 3

# ガウス・ジョルダン法 mod 3
def solve_mod3_system(A, b):
    A = A.copy()
    b = b.copy()
    n = len(b)
    aug = np.hstack([A, b.reshape(-1, 1)])

    for col in range(n):
        pivot = None
        for row in range(col, n):
            if aug[row][col] % 3 != 0:
                pivot = row
                break
        if pivot is None:
            continue
        if pivot != col:
            aug[[col, pivot]] = aug[[pivot, col]]
        inv = pow(int(aug[col][col]), -1, 3)
        aug[col] = (aug[col] * inv) % 3
        for row in range(n):
            if row != col and aug[row][col] != 0:
                factor = aug[row][col]
                aug[row] = (aug[row] - factor * aug[col]) % 3
    return aug[:, -1] % 3

# 色を1ステップ更新する関数（表示用）
def apply_step(state, panel):
    new_state = state[:]
    for neighbor in neighbors[panel]:
        new_state[neighbor] = (new_state[neighbor] + 1) % 3
    return new_state

# 表示用
def color_str(c): return ['赤', '青', '黄'][c]

def print_state(state):
    print("状態: " + ' '.join(color_str(c) for c in state))

# 踏む順序を生成（足踏みなし）
def find_order(steps):
    def backtrack(path, steps_left, prev):
        if sum(steps_left) == 0:
            return path
        for i in range(6):
            if steps_left[i] > 0 and i != prev:
                new_steps = steps_left[:]
                new_steps[i] -= 1
                result = backtrack(path + [i], new_steps, i)
                if result:
                    return result
        return None
    return backtrack([], steps, -1)

# メイン
def main():
    print("初期状態の色を6つ入力してください（赤・青・黄）:")
    user_input = input("例: 黄 青 青 赤 赤 赤\n> ").split()
    if len(user_input) != 6 or any(c not in color_map for c in user_input):
        print("エラー: 有効な6色（赤・青・黄）を指定してください。")
        return

    initial = [color_map[c] for c in user_input]
    target = [0] * 6  # すべて赤にしたい
    b = (np.array(target) - np.array(initial)) % 3
    steps = solve_mod3_system(A, b)

    print("\n各パネルを踏む回数:")
    for i, s in enumerate(steps):
        print(f"パネル{panel_names[i]} を {s} 回")

    order = find_order(list(steps))
    if order is None:
        print("\n足踏みなしの順序では解決できません。")
        return

    print("\n踏む順序と途中経過:")
    current = initial[:]
    print_state(current)
    for i, p in enumerate(order):
        current = apply_step(current, p)
        print(f"{i+1}. パネル{panel_names[p]} を踏む → ", end="")
        print_state(current)

    if all(c == current[0] for c in current):
        print("\n🎉 全てのパネルが同じ色（" + color_str(current[0]) + "）になりました！")
    else:
        print("\n⚠️ 色が揃いませんでした。解に誤りがある可能性があります。")

# 実行
if __name__ == "__main__":
    main()


初期状態の色を6つ入力してください（赤・青・黄）:
例: 黄 青 青 赤 赤 赤
> 黄 青 青 赤 赤 赤

各パネルを踏む回数:
パネルA を 1 回
パネルB を 0 回
パネルC を 1 回
パネルD を 2 回
パネルE を 2 回
パネルF を 1 回

踏む順序と途中経過:
状態: 黄 青 青 赤 赤 赤
1. パネルA を踏む → 状態: 黄 黄 青 赤 赤 青
2. パネルC を踏む → 状態: 黄 赤 青 青 赤 青
3. パネルD を踏む → 状態: 黄 赤 黄 青 青 青
4. パネルE を踏む → 状態: 黄 赤 黄 黄 青 黄
5. パネルD を踏む → 状態: 黄 赤 赤 黄 黄 黄
6. パネルE を踏む → 状態: 黄 赤 赤 赤 黄 赤
7. パネルF を踏む → 状態: 赤 赤 赤 赤 赤 赤

🎉 全てのパネルが同じ色（赤）になりました！
